In [1]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [15]:
import cv2
import numpy as np
import os
import sys

# --- 1. CÀI ĐẶT ---
# Đặt 2 ảnh vân tay (RÕ NÉT) của bạn ở đây
# Ảnh này coi như là ảnh đã đăng ký trong "CSDL"
IMAGE_DATABASE = 'van 4.jpg' 
# Ảnh này là ảnh bạn đưa vào để "check"
IMAGE_TO_CHECK = 'van 4.jpg'

# Ngưỡng (Threshold): 
# Cần bao nhiêu % điểm khớp để coi là "GIỐNG NHAU"?
# (Thường 15-20% là đủ tốt)
MATCH_THRESHOLD_PERCENT = 80

# --- 2. HÀM CHÍNH ---

def compare_fingerprints(img_path1, img_path2, threshold_percent):
    """
    So sánh hai ảnh vân tay bằng thuật toán ORB Feature Matching.
    """
    
    # --- A. TẢI ẢNH ---
    print(f"[INFO] Đang tải ảnh...")
    if not os.path.isfile(img_path1):
        print(f"[LỖI] Không tìm thấy ảnh CSDL: {img_path1}")
        return
    if not os.path.isfile(img_path2):
        print(f"[LỖI] Không tìm thấy ảnh để check: {img_path2}")
        return

    # Tải ảnh ở chế độ ảnh xám (grayscale)
    img1 = cv2.imread(img_path1, cv2.IMREAD_GRAYSCALE)
    img2 = cv2.imread(img_path2, cv2.IMREAD_GRAYSCALE)

    if img1 is None or img2 is None:
        print("[LỖI] Không thể đọc một trong hai ảnh.")
        return

    # --- B. TÌM ĐẶC ĐIỂM (FEATURES) BẰNG ORB ---
    # 1. Khởi tạo bộ phát hiện ORB
    # nfeatures=1000: Tìm tối đa 1000 điểm đặc biệt
    orb = cv2.ORB_create(nfeatures=1000)

    # 2. Tìm "Keypoints" (Vị trí) và "Descriptors" (Mô tả) cho cả 2 ảnh
    # Keypoint là điểm (x,y), Descriptor là "bản mã" mô tả điểm đó
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)

    if des1 is None or des2 is None:
        print("[LỖI] Không tìm thấy đủ đặc điểm trên một trong hai ảnh.")
        return

    # --- C. ĐỐI SÁNH (MATCHING) ---
    # 1. Khởi tạo bộ đối sánh (Brute-Force Matcher)
    # cv2.NORM_HAMMING: Dùng cho ORB
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

    # 2. Tìm các cặp khớp nhất (k=2: tìm 2 cặp gần nhất)
    matches = bf.knnMatch(des1, des2, k=2)
    
    if not matches or len(matches[0]) < 2:
        print("[LỖI] Không tìm thấy cặp nào để so sánh.")
        return

    # --- D. LỌC KẾT QUẢ (RATIO TEST) ---
    # Lọc ra các kết quả "tốt" (Lowe's ratio test)
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    # --- E. TÍNH KẾT QUẢ ---
    num_good_matches = len(good_matches)
    # Tính % dựa trên số keypoint của ảnh GỐC (img1)
    percent_match = (num_good_matches / len(kp1)) * 100

    print(f"\n--- KẾT QUẢ SO SÁNH ---")
    print(f"Tổng điểm đặc biệt (ảnh gốc):   {len(kp1)}")
    print(f"Tổng điểm đặc biệt (ảnh mới):    {len(kp2)}")
    print(f"Số điểm khớp tốt tìm thấy:     {num_good_matches}")
    print(f"Tỷ lệ khớp:                   {percent_match:.2f}%")

    if percent_match >= threshold_percent:
        print(f"KẾT LUẬN: GIỐNG NHAU (Vì {percent_match:.2f}% >= {threshold_percent}%)")
    else:
        print(f"KẾT LUẬN: KHÁC NHAU (Vì {percent_match:.2f}% < {threshold_percent}%)")

    # --- F. (TÙY CHỌN) VẼ KẾT QUẢ RA ẢNH ---
    # Vẽ các đường nối giữa các điểm khớp
    img_matches = cv2.drawMatches(img1, kp1, img2, kp2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
    
    # Resize ảnh kết quả cho dễ nhìn
    h, w = img_matches.shape[:2]
    if h > 800:
        scale = 800 / h
        img_matches_resized = cv2.resize(img_matches, (int(w * scale), int(h * scale)))
    else:
        img_matches_resized = img_matches
        
    cv2.imshow("Ket qua so sanh (Dung ORB)", img_matches_resized)
    print("\n[INFO] Nhấn phím bất kỳ trên cửa sổ ảnh để thoát...")
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# --- 3. HÀM CHÍNH ĐỂ CHẠY ---
def main():
    compare_fingerprints(IMAGE_DATABASE, IMAGE_TO_CHECK, MATCH_THRESHOLD_PERCENT)

if __name__ == "__main__":
    main()

[INFO] Đang tải ảnh...

--- KẾT QUẢ SO SÁNH ---
Tổng điểm đặc biệt (ảnh gốc):   477
Tổng điểm đặc biệt (ảnh mới):    477
Số điểm khớp tốt tìm thấy:     477
Tỷ lệ khớp:                   100.00%
KẾT LUẬN: GIỐNG NHAU (Vì 100.00% >= 80%)

[INFO] Nhấn phím bất kỳ trên cửa sổ ảnh để thoát...


In [104]:
import zipfile
import os

zip_file_name = 'van tay.zip'

if os.path.exists(zip_file_name):
    print(f"Đang giải nén file {zip_file_name}...")

    # 'r' = read (đọc), '.' = giải nén ra thư mục hiện tại
    with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
        zip_ref.extractall('.') 

    print(f"Đã giải nén xong! Bạn sẽ thấy thư mục 'van tay'.")
    
else:
    print(f"[LỖI] Không tìm thấy file {zip_file_name}. Bạn đã upload nó chưa?")

Đang giải nén file van tay.zip...
Đã giải nén xong! Bạn sẽ thấy thư mục 'van tay'.


In [28]:
import cv2
import numpy as np
import os
import sys

# --- 1. CÀI ĐẶT ---
# Thư mục chứa "data vân tay riêng"
DATASET_PATH = 'van tay' 
# Ảnh "vân tay tự chụp"
IMAGE_TO_CHECK = 'NM1.tif'

# Ngưỡng (Threshold):
# Cần TỐI THIỂU bao nhiêu ĐIỂM KHỚP để coi là "GIỐNG NHAU"?
MIN_MATCH_COUNT = 15 # Giữ nguyên 15, bạn có thể tăng lên 20-25 nếu muốn

# --- HÀM MỚI: DÙNG ĐỂ LÀM RÕ ẢNH VÂN TAY ---
def process_image(img_gray):
    """
    Áp dụng Adaptive Thresholding để làm rõ vân tay.
    """
    if img_gray is None:
        return None
        
    # Làm mờ nhẹ để giảm nhiễu (noise)
    img_blur = cv2.GaussianBlur(img_gray, (5, 5), 0)
    
    # Kỹ thuật "thần kỳ" là ở đây:
    # Biến ảnh xám (vân tay mờ) thành ảnh nhị phân (vân tay rõ nét)
    img_binary = cv2.adaptiveThreshold(
        img_blur, 
        255, # Giá trị tối đa (màu trắng)
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, # Phương pháp
        cv2.THRESH_BINARY_INV, # Đảo ngược: vân tay = TRẮNG, nền = ĐEN
                               # (ORB thích các điểm đặc biệt màu trắng)
        11, # Kích thước vùng lân cận (phải là số lẻ)
        2   # Hằng số C (trừ đi từ giá trị trung bình)
    )
    
    return img_binary
# --- KẾT THÚC HÀM MỚI ---


# --- 2. HÀM "HUẤN LUYỆN" (TẢI CSDL) - ĐÃ SỬA ---
def load_dataset_descriptors(path):
    """
    Tải tất cả ảnh, XỬ LÝ ẢNH, tính ORB và lưu vào list.
    """
    print(f"[INFO] Đang tải CSDL vân tay từ: {path}")
    known_descriptors = []
    
    orb = cv2.ORB_create(nfeatures=1000)
    
    if not os.path.exists(path):
        print(f"[LỖI] Không tìm thấy thư mục CSDL: {path}")
        sys.exit()

    # Quét thư mục 'van tay' (bản đơn giản, không quét thư mục con)
    for filename in os.listdir(path):
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp','.tif','.tiff')):
            continue # Bỏ qua các file không phải ảnh
            
        try:
            name = os.path.splitext(filename)[0]
            
            img_path = os.path.join(path, filename)
            img_gray = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            
            if img_gray is None:
                print(f"[CẢNH BÁO] Bỏ qua file: {filename} (không thể đọc)")
                continue
                
            # *** SỬA ĐỔI: XỬ LÝ ẢNH TRƯỚC KHI TÍNH ORB ***
            img_processed = process_image(img_gray)
            
            if img_processed is None:
                continue
                
            kp, des = orb.detectAndCompute(img_processed, None)
            
            if des is None:
                print(f"[CẢNH BÁO] Bỏ qua file: {filename} (không tìm thấy đặc điểm sau khi xử lý)")
                continue
                
            # Lưu (tên, kp, des) VÀ ảnh GỐC (img_gray) để vẽ
            known_descriptors.append((name, kp, des, img_gray)) 
            print(f"  -> Đã xử lý: {name} (tìm thấy {len(kp)} điểm)")
            
        except Exception as e:
            print(f"[LỖI] Xử lý file {filename} thất bại: {e}")
            
    print(f"[INFO] Đã tải xong CSDL ({len(known_descriptors)} người dùng).")
    return known_descriptors

# --- 3. HÀM SO SÁNH (TÌM KHỚP NHẤT) - ĐÃ SỬA ---
def find_best_match(image_to_check_path, known_descriptors, min_threshold):
    """
    So sánh 1 ảnh (đã qua xử lý) với TẤT CẢ CSDL đã tải.
    """
    
    # --- A. Tải và xử lý ảnh cần check ---
    if not os.path.isfile(image_to_check_path):
        print(f"[LỖI] Không tìm thấy ảnh cần check: {image_to_check_path}")
        return
        
    img_to_check_gray = cv2.imread(image_to_check_path, cv2.IMREAD_GRAYSCALE)
    if img_to_check_gray is None:
        print(f"[LỖI] Không thể đọc ảnh cần check.")
        return

    # *** SỬA ĐỔI: XỬ LÝ ẢNH CẦN CHECK (ẢNH MỜ) ***
    print("[INFO] Đang xử lý ảnh cần check (ảnh mờ)...")
    img_to_check_processed = process_image(img_to_check_gray)
    
    if img_to_check_processed is None:
        print("[LỖI] Xử lý ảnh cần check thất bại.")
        return

    orb = cv2.ORB_create(nfeatures=1000)
    # Tính ORB trên ảnh ĐÃ XỬ LÝ
    kp_new, des_new = orb.detectAndCompute(img_to_check_processed, None)
    
    if des_new is None:
        print("[LỖI] Không tìm thấy đặc điểm nào trên ảnh cần check (kể cả sau khi xử lý).")
        return

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
    
    # --- B. Vòng lặp so sánh với CSDL ---
    print("\n[INFO] Đang so sánh ảnh đã xử lý với CSDL...")
    
    best_score = 0
    best_name = "Khong ro"
    best_match_img = None
    best_kp = None
    best_good_matches = []

    # (name, kp_known, des_known, img_known_ORIGINAL)
    for (name, kp_known, des_known, img_known_original) in known_descriptors:
        try:
            matches = bf.knnMatch(des_known, des_new, k=2) # So sánh des (đã xử lý)
            good_matches = []
            for m, n in matches:
                # Kiểm tra n.distance > 0 để tránh lỗi chia cho 0
                if n.distance > 0 and m.distance < 0.75 * n.distance:
                    good_matches.append(m)
            
            num_good_matches = len(good_matches)
            print(f"  -> So với '{name}': tìm thấy {num_good_matches} điểm khớp.")
            
            if num_good_matches > best_score:
                best_score = num_good_matches
                best_name = name
                best_match_img = img_known_original # Lưu ảnh GỐC để vẽ
                best_kp = kp_known
                best_good_matches = good_matches
                
        except Exception as e:
            # Bắt lỗi chung, bao gồm cả lỗi 'k=2'
            print(f"[CẢNH BÁO] Lỗi khi so sánh với {name}: {e}")
            continue

    # --- C. In kết quả cuối cùng ---
    print(f"\n--- KẾT QUẢ CUỐI CÙNG ---")
    print(f"Tìm thấy khớp nhất là: {best_name}")
    print(f"Số điểm khớp cao nhất: {best_score}")

    if best_score >= min_threshold:
        print(f"KẾT LUẬN: GIỐNG NHAU (Vì {best_score} >= {min_threshold})")
        
        # Vẽ kết quả: (Ảnh GỐC CSDL) so với (Ảnh GỐC CẦN CHECK)
        img_result = cv2.drawMatches(
            best_match_img, best_kp, 
            img_to_check_gray, kp_new, # Dùng ảnh gốc và kp_new
            best_good_matches, None, 
            flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )
        
        h, w = img_result.shape[:2]
        if h > 800:
            scale = 800 / h
            img_result = cv2.resize(img_result, (int(w * scale), int(h * scale)))
            
        cv2.imshow(f"Ket qua: Khop voi {best_name}", img_result)
        
    else:
        print(f"KẾT LUẬN: KHÁC NHAU (Vì {best_score} < {min_threshold})")
        cv2.imshow("Ket qua: Khong khop", img_to_check_gray)

    print("\n[INFO] Nhấn phím bất kỳ trên cửa sổ ảnh để thoát...")
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# --- 4. HÀM CHÍNH ĐỂ CHẠY ---
def main():
    known_data = load_dataset_descriptors(DATASET_PATH)
    if not known_data:
        print("[LỖI] CSDL rỗng. Dừng chương trình.")
        return
    find_best_match(IMAGE_TO_CHECK, known_data, MIN_MATCH_COUNT)

if __name__ == "__main__":
    main()

[INFO] Đang tải CSDL vân tay từ: van tay
  -> Đã xử lý: Duong Ngo Nhat Minh (1) (tìm thấy 1000 điểm)
  -> Đã xử lý: Duong Ngo Nhat Minh (10) (tìm thấy 1000 điểm)
  -> Đã xử lý: Duong Ngo Nhat Minh (11) (tìm thấy 1000 điểm)
  -> Đã xử lý: Duong Ngo Nhat Minh (12) (tìm thấy 1000 điểm)
  -> Đã xử lý: Duong Ngo Nhat Minh (13) (tìm thấy 1000 điểm)
  -> Đã xử lý: Duong Ngo Nhat Minh (14) (tìm thấy 1000 điểm)
  -> Đã xử lý: Duong Ngo Nhat Minh (15) (tìm thấy 1000 điểm)
  -> Đã xử lý: Duong Ngo Nhat Minh (16) (tìm thấy 1000 điểm)
  -> Đã xử lý: Duong Ngo Nhat Minh (17) (tìm thấy 1000 điểm)
  -> Đã xử lý: Duong Ngo Nhat Minh (18) (tìm thấy 1000 điểm)
  -> Đã xử lý: Duong Ngo Nhat Minh (2) (tìm thấy 1000 điểm)
  -> Đã xử lý: Duong Ngo Nhat Minh (3) (tìm thấy 1000 điểm)
  -> Đã xử lý: Duong Ngo Nhat Minh (4) (tìm thấy 1001 điểm)
  -> Đã xử lý: Duong Ngo Nhat Minh (5) (tìm thấy 1000 điểm)
  -> Đã xử lý: Duong Ngo Nhat Minh (6) (tìm thấy 1000 điểm)
  -> Đã xử lý: Duong Ngo Nhat Minh (7) (tìm thấy 1